In [29]:
type(filter_abs_temp)

numpy.ndarray

In [4]:
import pandas as pd
import numpy as np
import scipy
from scipy import ndimage

import os, errno

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec


# define the width and length arrays 

rect_spc_x=0.001;
rect_spc_y=0.001;

rect_array_x = np.zeros(54)
rect_array_y = np.zeros(54)

for ix in range(54):
    rect_array_x[ix]=(rect_spc_x*(ix+1))/2

for iy in range(54):
    rect_array_y[iy]=(rect_spc_y*(iy+1))/2

    
#plt.plot(range(54),rect_array_x,'.')
#plt.show()





# load, slice, and reshape 2D image into to 1D array 
import sys
sys.path.append('/home/j2/home/j2/ml/env/lib/python3.5/site-packages')
from numpy import unravel_index

feature_star = 0 
feature_end = 3 # not inclusive

import cv2
import argparse

from sklearn.model_selection import cross_val_score, train_test_split
from sklearn import datasets
from sklearn import svm
from sklearn.model_selection import ShuffleSplit


flag_plot_gene = 2
xlennum=42
ylennum=42

xrange = range(2,xlennum+1,4)
yrange = range(2,ylennum+1,4)
rect_ixx = np.zeros(len(xrange))
rect_iyy = np.zeros(len(yrange))

image3D = np.zeros((401,401))
image2DLong = np.zeros(((401*401),1))
#image3D = []

clfscorelist = np.zeros(1)
cvscorelist = np.zeros(1)

#tholdmatrix = [0.02]
#tholdmatrix = [0.02,0.05,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.95,0.98]
#tholdmatrix = [0.02,0.05,0.1,0.2,0.3,0.4]
#tholdmatrix = [0.01,0.015,0.02,0.025,0.03,0.035,0.04,0.045,0.05]
tholdmatrix = [0.015]


cvscoremeanlist = np.zeros(1)

#random_statevalue = 1 



for thold in tholdmatrix:
    
    print('threshhold = ', thold)

    totalIndex = 0 
    directory = '/home/j2/Desktop/PythonTry/MLshm/oput/feature_star%dthold%03d' % (feature_star,thold*100)

    try:
        os.makedirs(directory)
    except OSError as e:
        if e.errno != errno.EEXIST:
            raise

    nnum = 401 

    kernelx = 15
    kernely = 15
    kernaltotal = kernelx * kernely

    kernel = np.ones((kernelx,kernely),np.float32)/kernaltotal

    df = pd.read_csv('sumValuesThrhPer20.csv')
    dfXtemp = df[['x (mm)','y (mm)']]

    X = np.zeros((1,2))
    Y = np.zeros((1,6))


    for ixx in xrange: #range(2,10+1,4):#
        for iyy in yrange: # range(2,6+1,4):#
            rect_ixx = rect_array_x[ixx]
            rect_iyy = rect_array_y[iyy]

            totalIndex += 1 
            Dxmm = int(round(2*rect_ixx*1e3-1))    
            Dymm = int(round(2*rect_iyy*1e3-1))

            parafilename = '/home/j2/Desktop/PythonTry/MLshm/MLRTMwithExperiment/rectx%03dy%03d/iter00000/bnzlcc.csv' % (Dxmm,Dymm)
            tempdf = pd.read_csv(parafilename,header=None)


            tempdfabs = abs(tempdf.values)
            filter_abs = cv2.filter2D(tempdfabs,-1,kernel)
            filter_abs_temp = filter_abs
            maxfilter_abs = filter_abs.max().max()
            filter_abs_temp[filter_abs < thold*(dfY[['max value']].max().values)] = 0
            filter_abs_temp[:,:130] = 0


            if flag_plot_gene == 1: 
                imagfilename = '/home/j2/Desktop/PythonTry/MLshm/oput/feature_star%dthold%03d/filter%d.png' % (feature_star,(thold*100),totalIndex)
                fig = plt.figure()
                plt.imshow(ndimage.rotate(filter_abs_temp,90))

                plt.title('Dx = %d mm   Dy= %d mm' % (Dxmm,Dymm))
                plt.savefig(imagfilename,format='png', dpi=300)
                #plt.show()
                plt.close()

            LL = np.zeros([nnum,nnum])
            #LL[filter_abs > thold*maxfilter_abs] = 1
            LL[filter_abs > thold*(dfY[['max value']].max().values)] = 1


            P_area_num=LL.sum().sum()
            P_width = np.sum(LL,axis=0).max()
            P_length = np.sum(LL,axis=1).max()
            Pxy = unravel_index(filter_abs_temp.argmax(), filter_abs_temp.shape)  
            onerow = np.array([P_area_num,P_width,P_length,Pxy[0],Pxy[1],maxfilter_abs]).reshape((1,6))
            onerowX = np.array([Dxmm,Dymm]).reshape((1,2))
            X = np.append(X,onerowX,axis=0)
            Y = np.append(Y,onerow,axis=0)


            #temp = tempdf.values
            #image3D = np.dstack((image3D, temp)) 


            #tempLong= temp.reshape(401*401,1)
            #image2DLong = np.dstack((image2DLong,tempLongrandom_state))

        # get rid of the starting image         
        #imagefinal3D = image3D[:,:,1:]
        #imagefinal2DLong = np.transpose(image2DLong[:,:,1:].reshape(160801,121))

    Yfinal = Y[1:,:]
    Xfinal = X[1:,:]
    X = Xfinal
    colname = ["Area (mm^2)","Length (mm)","Width (mm)","x_center","y_center","max value"]
    dfY = pd.DataFrame(Yfinal, columns=colname)
    file_name ='iput/filterdfYthold%03d.csv' % (thold*100)
    dfY.to_pickle(file_name)

    #print(P_area_num,P_width,P_length2,Pxy[0],Pxy[1],maxfilter_abs)
    print('threshold = ',thold*(dfY[['max value']].max().values))

    file_name ='iput/filterdfYthold%03d.csv' % (thold*100)
    dfY = pd.read_pickle(file_name)
    #X=dfX.values
    Y=dfY.values


    # select the first three attributes of the thresholded image: area, width, length 
    Y3 = Y[:,feature_star:feature_end]


    X_train, X_test, Y3_train, Y3_test = train_test_split(X, Y3, test_size=0.02, random_state=random_statevalue)
    #print(X_train.shape)
    #print(X_test.shape)
    #print(Y3_train.shape)
    #print(Y3_test.shape)


    from sklearn import linear_model
    # ordinary least squares
    clf = linear_model.LinearRegression()
    # univariate
    clf.fit(X_train,Y3_train)
    clf.score(X_test, Y3_test) 


    def NNpredcheck(x,y,model):
        Yt1 = model.predict(np.array([[x, y]]))
        return Yt1

    textclfscore = ' clf score : ' + np.array_str(clf.score(X_test, Y3_test)) + '\n'
    clfscorestr = np.array_str(clf.score(X_test, Y3_test)) + '\n'
    text35 = 'output (3 5): ' + np.array_str(NNpredcheck(3,5,clf)) + '\n'
    headline = 'feature_star %d thold %d percent  \n' % (feature_star,thold*100)
    #NNpredcheck(3,5,clf)

    outputfilename ='oput/NN3parafeature_star%dthold%03d.csv' % (feature_star,thold*100)

    with open(outputfilename, "w") as f:
        f.write(headline)
        f.write(textclfscore)
        f.write(clfscorestr)
        f.write(text35)


    #indexnumber = 1
    f= open(outputfilename,"a+")
    for irad in range(2):
        indexnumber = irad
        #print('index number is =', indexnumber)
        str1 =  'width and length = ' + np.array_str(X_test[indexnumber,:].reshape(1,-1)) +'\n'
        f.write(str1)
        print('width and length = ', X_test[indexnumber,:].reshape(1,-1) )
        str1 =  'predicted = ' + np.array_str(clf.predict(X_test[indexnumber,:].reshape(1,-1))) +'\n'
        f.write(str1)
        print('predicted = ', clf.predict(X_test[indexnumber,:].reshape(1,-1) ))
        str1 =  'labeled = ' + np.array_str(Y3_test[indexnumber,:]) +'\n'
        f.write(str1)
        print('labeled = ', Y3_test[indexnumber,:])
        str1 = 'Ratio = ' + np.array_str(clf.predict(X_test[indexnumber,:].reshape(1,-1) )/Y3_test[indexnumber,:]) + '\n'
        f.write(str1)
        print('Ratio = ', clf.predict(X_test[indexnumber,:].reshape(1,-1) )/Y3_test[indexnumber,:]) 

    f.close()

    print(textclfscore)
    cv = ShuffleSplit(n_splits=121, test_size=0.02, random_state=random_statevalue)
    cvscores = cross_val_score(clf, X, Y3, cv=cv)
    #cvscores = cross_val_score(clf, X, Y3, cv=60)
    print('CV score are', cvscores)
    print('Mean of cv_scores are', cvscores.mean())

    #textclfscore
    Y = np.append(Y,onerow,axis=0)

    clfscorelist = np.append(clfscorelist,clf.score(X_test, Y3_test)) 
    cvscorelist = np.append(cvscorelist,cvscores.mean()) 

clfscorelistFinal = clfscorelist[1:]
cvscorelistFinal = cvscorelist[1:]

clfscoredict = pd.Series(clfscorelistFinal,index=tholdmatrix)
cvscoredict = pd.Series(cvscorelistFinal,index=tholdmatrix)

cvscoremeanlist = np.append(cvscoremeanlist,cvscores.mean()) 




cvscoremeanlistFinal = cvscoremeanlist[1:]
cvscoremeandict = pd.Series(cvscoremeanlistFinal,index=tholdmatrix)


fig = plt.figure()
plt.plot(cvscoremeandict, 'b-')
#plt.plot(cvscoredict, 'r-')
plt.title('Feasure start %d, end %d, threshold %d' % (feature_star,feature_end, thold))
plt.xlabel('threshold')
plt.ylabel('score for the test dataset')
plt.savefig(imagfilename,format='png', dpi=300)
plt.legend(['one split', 'averaged CV score'], loc=4) # 
plt.show()
plt.close()




threshhold =  0.015


NameError: name 'dfY' is not defined

In [3]:
(dfY[['max value']].max().values)

NameError: name 'dfY' is not defined

In [ ]:
imagfilename = '/home/j2/Desktop/PythonTry/MLshm/oput/scorelistfeature_star%d.png' % (feature_star)
fig = plt.figure()
plt.plot(clfscoredict, 'b-')
plt.plot(cvscoredict, 'r-')
plt.title('Feasure start %d, end %d, random_state %d' % (feature_star,feature_end,random_statevalue))
plt.xlabel('threshold')
plt.ylabel('score for the test dataset')
plt.savefig(imagfilename,format='png', dpi=300)
plt.legend(['one split', 'averaged CV score'], loc=4) # 
plt.show()
plt.close()
 

In [59]:
### visualize the input data

# from mpl_toolkits.mplot3d import Axes3D
# import matplotlib.pyplot as plt
# import numpy as np

# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')

# n = 100

# ax.set_xlabel('X Label')
# ax.set_ylabel('Y Label')
# ax.set_zlabel('Z Label')

# import pandas as pd
# from pandas import DataFrame
# import matplotlib.pyplot as plt
# from mpl_toolkits.mplot3d import Axes3D

# def zeplot(dfX,dfY,titlename):
#     threedee = plt.figure().gca(projection='3d')
#     plt.title(titlename)
#     threedee.scatter(dfX[['x (mm)']], dfX[['y (mm)']],dfY[[titlename]])
#     threedee.set_xlabel('x (mm)')
#     threedee.set_ylabel('y (mm)')
#     threedee.set_zlabel(titlename)
#     plt.show()


    
#for iy in range(len(dfY.columns)):
#    zeplot(dfX,dfY,dfY.columns[iy])  

In [67]:
clfscoredict

0.02    0.989558
0.05    0.952194
0.10    0.960157
0.20    0.971835
0.30    0.958093
0.40    0.954159
dtype: float64

In [18]:
dfY[['max value']].max()

max value    0.165651
dtype: float64

In [82]:
#num2 = 5 
#for i in range(42*num2):
#    for j in range(42*num2):
#        print((i+1)/num2,(j+1)/num2,'=',NNpredcheck((i+1)/num2,(j+1)/num2,clf)) 

In [34]:
# #indexnumber = 1
# for irad in range(2):
#     indexnumber = irad
#     #print('index number is =', indexnumber)
      
#     print('width and length = ', X_test[indexnumber,:].reshape(1,-1) )
#     print('predicted = ', clf.predict(X_test[indexnumber,:].reshape(1,-1) ))
#     print('labeled = ', Y3_test[indexnumber,:])
#     print('Ratio = ', clf.predict(X_test[indexnumber,:].reshape(1,-1) )/Y3_test[indexnumber,:]) 

In [22]:
thold*(dfY[['max value']].max()).values

array([ 0.00331301])